In [1]:
# %load my_imports.py
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import csv

import datetime
import time

import json



In [2]:
# Dict writer n reader
def write_to_file(name, dict_object):
    with open(name, 'wb') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in dict_object.items():
           writer.writerow([key, value])
        
def load_from_file(file_name):
    dict_object = {}
    df = pd.read_csv(file_name, names = ['key', 'value'])
    for idx in df.index:
        key = df.key[idx]
        value = [int(value) for value in df.value[idx][1:-1].split(',')]
        dict_object.update({key: value})
    return dict_object

def load_lists(file_name):
    dict_object = {}
    df = pd.read_csv(file_name, names = ['value', 'key'])
    for idx in df.index:
        dict_object.update({df.key[idx]: df.value[idx]})
    return dict_object

def load_lists_reversed(file_name):
    dict_object = {}
    df = pd.read_csv(file_name, names = ['key', 'value'])
    for idx in df.index:
        dict_object.update({df.key[idx]: df.value[idx]})
    return dict_object

def load_pagerank(file_name):
    dict_object = {}
    df = pd.read_csv(file_name, names = ['key', 'value'])
    for idx in df.index:
        dict_object.update({int(df.key[idx]): df.value[idx]})
    return dict_object

In [3]:
movie_list = load_lists('trimmed_special/movie_index_dict_trimmed.csv')
actor_list = load_lists('trimmed_special/actor_index_dict_trimmed.csv')
movie_actor_dict_int_int = load_from_file('trimmed_special/movie_actor_dict_int_int_trimmed.csv')
actor_movie_dict_int_int = load_from_file('trimmed_special/actor_movie_dict_int_int_trimmed.csv')
# actor_list_reversed = load_lists_reversed('10 networks/actor_dict.csv')
# movie_list_reversed = load_lists_reversed('10 networks/movie_dict.csv')

In [4]:
movie_list

{'Rascoala (1965)': 116034,
 'Auf der gr\xfcnen Wiese (1953)': 2,
 'Aleksandr Popov (1949)': 399000,
 'Parties chaudes (1979)': 408852,
 'Country (2000)': 233719,
 'Bond of Fear (1956)': 291799,
 'Hometown Honeys (1986)': 291795,
 'Gaiking II (2011)': 7,
 "La corda d'acciaio (1954)": 350146,
 'Medusa (2015/IV)': 9,
 'Debika (1987)': 10,
 'Alex in Wonderland (1970)': 233723,
 'Do Outro Lado do Mundo (2008)': 14,
 'Insaaf Main Karoonga (1985)': 16,
 'Un hombre como los dem\xe1s (1974)': 116048,
 'O Pr\xedncipe (2002)': 174710,
 'Les marins perdus (2003)': 70978,
 'Los siete locos (1973)': 105,
 'Kin chan no Cinema Jack (1993)': 116053,
 'One Day Like Rain (2007)': 174712,
 'When Justice Fails (1999)': 174713,
 'Mahapurush (1965)': 24,
 'Julietta (1953)': 418670,
 'Tinseltown (1980)': 115584,
 'Ang pumatay ng dahil sa iyo (1989)': 29,
 "Polites B' katigorias (1981)": 30,
 'Die Nackte und der Satan (1959)': 57889,
 'Aff\xe6ren Birte (1945)': 414165,
 'Rats (2000)': 291819,
 'Firstborn (198

In [5]:
# Community_dict
# somehow I lost 1 node 52670 --52669
with open ('others/membership.txt') as f:
    contents = f.readlines()
with open ('others/movie_index.txt') as f:
    lines = f.readlines()
contents = contents[1:]
lines = lines[1:]
com_dict = {}
for idx, com in zip(lines, contents):
    key = int(re.split(r"[~,\n\"]+", idx)[2])
    value = int(re.split(r"[~,\n]+", com)[1])
    com_dict.update({key: value})
len(com_dict)

52672

In [6]:
movie_list_reversed = dict((y,x) for x, y in movie_list.iteritems())

In [7]:
# genre dict
# insert entry that are in movie_list
with open ('movies/movie_genre.txt') as f:
    contents = f.readlines()
genre_dict = {}
for line in contents:
    content = re.split(r"[~\t\n]+", line)
    try:
        genre_dict.update({movie_list[content[0]]: content[-2]})
    except: pass
len(genre_dict)

46787

In [8]:
# reverse com_dict
com_dict_reversed = {}
for key in list(set(com_dict.values())):
    com_dict_reversed.update({key: []})
for movie in com_dict.keys():
    com_dict_reversed[com_dict[movie]].append(movie)

In [9]:
# Add a genre_index_list
genre_list = list(set(genre_dict.values()))
genre_list.append('unknown')
genre_index = [x for x in range(1, len(genre_list) + 1)]
genre_index_list = {}
for key, value in zip (genre_list, genre_index):
    genre_index_list.update({key: value})

In [10]:
# generate com_genre_dict
com_genre_dict = {}
for key in com_dict_reversed.keys():
    genres = []
    for movie in com_dict_reversed[key]:
        try:
            genres.append(genre_index_list[genre_dict[movie]])
        except: 
#             pass
            genres.append(genre_index_list['unknown'])
    com_genre_dict.update({key: genres})

In [11]:
size = [value for genre in com_genre_dict.values() for value in genre]
len(size)

52672

In [12]:
# reverse genre_index_list
genre_index_list_reverse = dict((y,x) for x, y in genre_index_list.iteritems())

In [13]:
# Generate community Analysis 
# Tag 20%
from collections import Counter
for key in com_genre_dict.keys():
    print ('Community %d' %key)
    tags = Counter(com_genre_dict[key]).most_common(5)
    for genre_pair in tags:
        if genre_pair[1] * 5 > len(com_genre_dict[key]):
#             print genre_pair[1]
#             print len(com_genre_dict[key])
            print('%s:%f' %(genre_index_list_reverse[genre_pair[0]], genre_pair[1]/float(len(com_genre_dict[key])))),
    print '\n'

Community 1
Drama:0.219311 

Community 2
Drama:0.218995 

Community 3
Drama:0.295918 

Community 4
Drama:0.346154 unknown:0.211538 

Community 5
Drama:0.379360 

Community 6
Romance:0.283333 Drama:0.216667 

Community 7
unknown:0.252747 Drama:0.222253 

Community 8
Drama:0.258569 

Community 9
Drama:0.308458 

Community 10
Drama:0.230553 Short:0.217247 

Community 11
Drama:0.285550 

Community 12
unknown:0.235094 Romance:0.217206 

Community 13
Drama:0.332398 Comedy:0.308555 

Community 14
Drama:0.363636 

Community 15
Comedy:0.345143 Drama:0.323429 

Community 16
Musical:0.450000 unknown:0.250000 

Community 17
Drama:0.454545 War:0.227273 

Community 18
unknown:0.513889 

Community 19
Drama:0.303093 Romance:0.204811 

Community 20
Drama:0.642105 

Community 21
Romance:0.335853 Drama:0.278618 

Community 22
Drama:0.271429 

Community 23
Drama:0.333333 Thriller:0.333333 

Community 24
Thriller:0.857143 



In [66]:
# print 5 nearest neighbors
target_movies = ['Batman v Superman: Dawn of Justice (2016)', 'Mission: Impossible - Rogue Nation (2015)', 'Minions (2015)']

for i in range(1, 4):
    print ('5 nearest neighbors of ' + target_movies[i-1]),
    print (genre_dict[movie_list[target_movies[i-1]]])
    with open ('others/core%d.txt' %i) as f:
        contents = f.readlines()
    with open ('others/weight%d.txt' %i) as f:
        lines = f.readlines() 
    temp_1 = [int(content.split('\n')[0][1:-1]) for content in contents]
    temp_2 = [float(content.split('\n')[0][1:-1]) for content in lines]
    df = pd.DataFrame(
        {'Name': temp_1,
         'Weight': temp_2}) 
    df.sort_values(by='Weight', ascending=False, inplace=True)
    top_5= df.Name[0:5]
    for j in top_5:
        print movie_list_reversed[j],
        print com_dict[j],
        print "/",
        print genre_dict[j]
    print('\n')

 5 nearest neighbors of Batman v Superman: Dawn of Justice (2016) Sci-Fi
Grain (2015) 8 / Drama
Rather to be Chosen (2015) 1 / Romance
Dial a Prayer (2015) 1 / Drama
Behind the Scenes of Horrorcore Hotel (2014) 1 / Short
The Butterfly Effect 3: Revelations (2009) 1 / Thriller


5 nearest neighbors of Mission: Impossible - Rogue Nation (2015) Thriller
The Fall of the Krays (2015) 1 / Crime
Cocktail (2012) 12 / Romance
Londongrad (????) 1 / Drama
Welcome to Karachi (2015) 12 / War
Solito (2013) 1 / Drama


5 nearest neighbors of Minions (2015) Family
Tokyo Mater (2008) 7 / Short
Crash Tag Team Racing (2005) 7 / Sport
King's Quest VIII: Mask of Eternity (1998) 7 / Fantasy
Star Wars: Racer Revenge (2002) 7 / Sci-Fi
Ratchet & Clank Future: Quest for Booty (2008) 7 / Sci-Fi




In [70]:
# load movie ratings
movie_ratings = {}
with open ('movies/movie_rating.txt') as f:
    contents = f.readlines()
for content in contents:
    line = filter(None, re.split(r"[~\t\n]+", content))
    movie_ratings.update({line[0]: float(line[-1])})
    
movie_int_ratings = {}
no_ratings = 0
for key in movie_actor_dict_int_int.keys():
    try:
        movie_int_ratings.update({key: [movie_ratings[movie_list_reversed[key]]]})
    except: 
        no_ratings +=1
        
print(len(movie_actor_dict_int_int))
print(no_ratings)
print(len(movie_int_ratings))

52673
14580
38093


In [110]:
# unweighted
# print ratings based on neighbors
target_movies = ['Batman v Superman: Dawn of Justice (2016)', 'Mission: Impossible - Rogue Nation (2015)', 'Minions (2015)']

for i in range(1, 4):
    with open ('others/core%d.txt' %i) as f:
        contents = f.readlines()
    with open ('others/weight%d.txt' %i) as f:
        lines = f.readlines() 
    temp_1 = [int(content.split('\n')[0][1:-1]) for content in contents]
    temp_2 = [float(content.split('\n')[0][1:-1]) for content in lines]
    df = pd.DataFrame(
        {'Name': temp_1,
         'Weight': temp_2}) 
    df.sort_values(by='Weight', ascending=False, inplace=True)
    top_15 = df.Name[0:15]
    rating = []
    for j in top_15:
        try: rating.append(movie_int_ratings[j][0])
        except: pass
    print ('rating of ' + target_movies[i-1]),
    print (': %f' %np.mean(rating)),
    print len(rating)
    print ('\n')

rating of Batman v Superman: Dawn of Justice (2016) : 6.283333 6


rating of Mission: Impossible - Rogue Nation (2015) : 6.185714 7


rating of Minions (2015) : 6.742857 14




In [131]:
# weighted
# print ratings based on neighbors
target_movies = ['Batman v Superman: Dawn of Justice (2016)', 'Mission: Impossible - Rogue Nation (2015)', 'Minions (2015)']
for i in range(1, 4):
    with open ('others/core%d.txt' %i) as f:
        contents = f.readlines()
    with open ('others/weight%d.txt' %i) as f:
        lines = f.readlines() 
    temp_1 = [int(content.split('\n')[0][1:-1]) for content in contents]
    temp_2 = [float(content.split('\n')[0][1:-1]) for content in lines]
    df = pd.DataFrame(
        {'Name': temp_1,
         'Weight': temp_2}) 
    df.sort_values(by='Weight', ascending=False, inplace=True)
    top_15 = df.Name[0:15]
    rating = []
    weight = []
    top_15_weight = df.Weight[0:15]
    for j, k in zip(top_15, top_15_weight):
        try: 
            rating.append(movie_int_ratings[j][0] * k)
            weight.append(k)
        except: pass
    score = np.mean(rating)/float(sum(weight))*len(weight)
    print ('rating of ' + target_movies[i-1]),
    print (': %f' %score)
    print ('\n')

rating of Batman v Superman: Dawn of Justice (2016) : 6.279793


rating of Mission: Impossible - Rogue Nation (2015) : 6.273461


rating of Minions (2015) : 6.828282




In [129]:
np.mean(rating)/float(sum(weight))

1.0466322315298646

In [113]:
top_15

112    456013
81     313051
85     341123
42     154812
41     152403
37     142293
65     268812
9       33271
97     401272
83     335573
82     316694
78     303695
94     390249
24      77331
84     340392
Name: Name, dtype: int64